<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li></ul></div>

# Intro
Notebook to run stable-diffusion, relying on the [Diffusers library](https://github.com/huggingface/diffusers).

TODOs:
- masking (need another model?)
- depth
- pipe.enable_attention_slicing()

In [ ]:
from pathlib import Path
from datetime import datetime
import json, os, sys
import cv2
import numpy as np
import torch
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

from diffusers_utils import load_pipelines, generate, diffusion_process_run
from diffusers_utils import save_config_to_file, Config

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

In [ ]:
opt = Config()
opt.model_path = 'D:/models/stable_diffusion/stable-diffusion-v1-4'
opt.scheduler = 'dpms' # ['lmsd', 'euler', 'ddim', 'dpms'] # TODO allow for dynamic change of scheduler

sd_pipe, img2img_pipe, inpaint_pipe = load_pipelines(opt, device)

batch_idx = 0

In [ ]:
outputs_path = Path.home() / 'Documents/generated_data/stable_diffusion'
batch_name = f'test' # name for subfolder and filenames  {datetime.now().strftime("%Y%m%d_%H%M%S")}_
width_height = [500, 500] 
opt.W, opt.H = width_height
opt.scale = 14 # strength of text prompt
opt.n_steps = 50 # number of diffusion steps
opt.seed = -1 # use -1 for random seed or specify number manually
opt.seamless = False  # whether to generate seamless images
opt.num_batch_images = 5 # how many images you want to generate in this batch

# Init image
opt.init_img = str('') # url or path to an image/video
opt.strength = 0.65 # from 0.0 to 1.0, strength of noise applied on the init image
opt.skip_frames = 2 # how many frames to skip if init image is a video

# Mask
opt.mask_path = None # url or path to an image/video that will be used for masking
opt.invert_mask = False

# Prompt
opt.prompt = """"""


opt.outdir = str(outputs_path / batch_name)
opt.process_config()

if opt.init_img is None:
    pipe = sd_pipe
else:
    if opt.mask_path is None:
        pipe = img2img_pipe
    else:
        pipe = inpaint_pipe
        
#pipe.enable_attention_slicing()    
diffusion_process_run(opt, pipe, batch_name, batch_idx)


In [ ]:
from PIL import Image
img = Image.open(str('')).convert('RGB')
img.thumbnail([600, 800], Image.ANTIALIAS)  # maintain aspect ratio
img.size